In [33]:
import pandas as pd
import numpy as np

Initial Data set

In [34]:
book = pd.read_csv("Data/BX-Books.csv", sep=";", encoding="latin-1", on_bad_lines="skip")
ratings = pd.read_csv("Data/BX-Book-Ratings.csv", sep=";", encoding="latin-1", on_bad_lines="skip")

C:\Users\Ashwanth\AppData\Local\Temp\ipykernel_17568\1768993721.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  book = pd.read_csv("Data/BX-Books.csv", sep=";", encoding="latin-1", on_bad_lines="skip")


Book Data

In [35]:
book.drop(columns=["Image-URL-S","Image-URL-M"], inplace=True)
book = book.rename(columns={
    "Book-Title": "title",
    "Book-Author": "author",
    "Year-Of-Publication": "year",
    "Publisher": "publisher",
    "Image-URL-L": "image_url",
})
book.head()

,ISBN,title,author,year,publisher,image_url
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...


In [36]:
book.shape

(271360, 6)

Ratings Data

In [37]:
ratings = ratings.rename(columns={
    "User-ID": "user_id",
    "Book-Rating": "rating",
})
ratings.head()

,user_id,ISBN,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [38]:
ratings.shape

(1149780, 3)

Taking the user_id how rated books more than 200 time

In [39]:
X = ratings['user_id'].value_counts() > 200
y = X[X].index
ratings = ratings[ratings['user_id'].isin(y)]
ratings.shape

(526356, 3)

Rated_Book Data


Merging the Ratings and Book data

In [42]:
rated_books = ratings.merge(book, on="ISBN")
rated_books.shape

(487671, 8)

In [43]:
rated_books.head()

,user_id,ISBN,rating,title,author,year,publisher,image_url
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
1,277427,0026217457,0,Vegetarian Times Complete Cookbook,Lucy Moll,1995,John Wiley &amp; Sons,http://images.amazon.com/images/P/0026217457.0...
2,277427,003008685X,8,Pioneers,James Fenimore Cooper,1974,Thomson Learning,http://images.amazon.com/images/P/003008685X.0...
3,277427,0030615321,0,"Ask for May, Settle for June (A Doonesbury book)",G. B. Trudeau,1982,Henry Holt &amp; Co,http://images.amazon.com/images/P/0030615321.0...
4,277427,0060002050,0,On a Wicked Dawn (Cynster Novels),Stephanie Laurens,2002,Avon Books,http://images.amazon.com/images/P/0060002050.0...


Number of Ratings per Book => num_of_ratings_per_book

In [70]:
num_of_ratings_per_book = rated_books.groupby("title")["rating"].count().reset_index()

In [71]:
num_of_ratings_per_book.rename(columns={
    "rating": "num_of_ratings"
}, inplace=True)
num_of_ratings_per_book.head()

,title,num_of_ratings
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


In [72]:
num_of_ratings_per_book.shape

(160269, 2)

Final Rating Data

In [74]:
final_ratings = rated_books.merge(num_of_ratings_per_book, on="title")

In [75]:
final_ratings.shape

(487671, 9)

In [76]:
final_ratings.head()

,user_id,ISBN,rating,title,author,year,publisher,image_url,num_of_ratings
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
1,277427,0026217457,0,Vegetarian Times Complete Cookbook,Lucy Moll,1995,John Wiley &amp; Sons,http://images.amazon.com/images/P/0026217457.0...,7
2,277427,003008685X,8,Pioneers,James Fenimore Cooper,1974,Thomson Learning,http://images.amazon.com/images/P/003008685X.0...,1
3,277427,0030615321,0,"Ask for May, Settle for June (A Doonesbury book)",G. B. Trudeau,1982,Henry Holt &amp; Co,http://images.amazon.com/images/P/0030615321.0...,1
4,277427,0060002050,0,On a Wicked Dawn (Cynster Novels),Stephanie Laurens,2002,Avon Books,http://images.amazon.com/images/P/0060002050.0...,13


Taking num_of_rating more than 50 ratings

In [77]:
final_ratings = final_ratings[final_ratings["num_of_ratings"] >= 50]
final_ratings.shape

(61853, 9)

Pre Processing Data

In [78]:
final_ratings.drop_duplicates(subset=["user_id", "ISBN"], inplace=True)
final_ratings.shape

(61853, 9)

Pivot Table

In [79]:
book_pivot = final_ratings.pivot_table(
    index="title",
    columns="user_id",
    values="rating")
book_pivot.head()

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Chance,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN
4 Blondes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84 Charing Cross Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN


In [80]:
book_pivot.fillna(0, inplace=True)
book_pivot.head()

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0


In [ ]:
book_names = book_pivot.index
book_names

Index(['1984', '1st to Die: A Novel', '2nd Chance', '4 Blondes',
       '84 Charing Cross Road', 'A Bend in the Road', 'A Case of Need',
       'A Child Called \It\": One Child's Courage to Survive"',
       'A Civil Action', 'A Cry In The Night',
       ...
       'Winter Solstice', 'Wish You Well', 'Without Remorse',
       'Wizard and Glass (The Dark Tower, Book 4)', 'Wuthering Heights',
       'Year of Wonders', 'You Belong To Me',
       'Zen and the Art of Motorcycle Maintenance: An Inquiry into Values',
       'Zoya', '\O\" Is for Outlaw"'],
      dtype='object', name='title', length=742)

use CSR Matrix ->> Compressed Spares Row

In [81]:
from scipy.sparse import csr_matrix
book_sparse = csr_matrix(book_pivot.values)

In [82]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm='brute')
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [83]:
def book_recommendation(book_name):
    book_id = np.where(book_pivot.index == book_name)[0][0]
    distances, suggestion = model.kneighbors(book_sparse[book_id], n_neighbors=6)
    for i in range(len(suggestion)):
        book = book_pivot.index[suggestion[i]]
        for j in book:
            print(j)

In [ ]:
name = "You Belong To Me"
book_recommendation(name)

A Bend in the Road
Exclusive
The Cradle Will Fall
No Safe Place
Last Man Standing
Family Album


save the model

In [88]:
import pickle
pickle.dump(model, open('artifact/model.pkl', 'wb'))
pickle.dump(book_names, open('artifact/book_names.pkl', 'wb'))
pickle.dump(final_ratings, open('artifact/final_rating.pkl', 'wb'))
pickle.dump(book_pivot, open('artifact/book_pivot.pkl', 'wb'))